<a href="https://colab.research.google.com/github/johyunkang/adp_certificate/blob/main/adp_stats_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADP 27회 통

Q1. 2년 전 제품 생산량이 100,000개, 1년 전 제품 생산량이 150,000개, 그 후 팩토리 기술의 상승으로 제품 생산량이 250,000개 되었을 때, 연평균 상승률의 대푯값을 구하시오. (반올림하여 소수점 아래 둘째자리까지 표기하시오.)

In [24]:
import numpy as np

past_2y = 100_000
past_1y = 150_000
now = 250_000

rt1 = (past_1y - past_2y)/ past_2y
print('1년전 상승률:', rt1 * 100, ' %')

rt2 = (now - past_1y) / now
print('올해 상승률:', rt2 * 100, ' %')
print('연평균 상승률:', (rt1+rt2)/2 * 100, '%')
ans = pow((250_000/100_000), (1/2)) - 1
print('연평균 상승률(CAGR):', round(ans * 100, 2))

1년전 상승률: 50.0  %
올해 상승률: 40.0  %
연평균 상승률: 45.0 %
연평균 상승률(CAGR): 58.11


In [23]:
print(round((pow((now/past_2y), (1/2)) - 1) * 100, 2), '%')

import statistics  as sts

n_list = [100_000, 150_000, 250_000]
rt1 = sts.mean(n_list[:2])
rt2 = sts.mean(n_list[1:])
print(rt1, rt2)
print(sts.mean(n_list))
geo_mean = sts.geometric_mean([rt1, rt2])
print(geo_mean)

print(n_list[:2], n_list[1:])


58.11 %
125000 200000
166666.66666666666
158113.88300841878
[100000, 150000] [150000, 250000]


#### NPV

In [36]:
cashflow = [-750000, 10000, 20000, 30000, 40000]
r = 0.05 # 할인률

import numpy as np
net_present_value = np.npv(r, cashflow)
print('NPV:', net_present_value)

<ipython-input-36-80a0f3b357cd>:5: DeprecationWarning: In accordance with NEP 32, the function npv was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  net_present_value = np.npv(r, cashflow)


RuntimeError: ignored

In [37]:
import numpy_financial

ModuleNotFoundError: ignored